In [ ]:
import shapely, \
    matplotlib,  \
    fiona,\
    numpy as np, \
    pandas as pd, \
    geopandas as gpd, \
    matplotlib.pyplot as plt, \
    geoplot as gplt, \
    mapclassify as mcs, \
    seaborn as sns, \
    pysal as ps,\
    libpysal as lps, \
    math

# import packages

from esda.moran import Moran
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
from libpysal.cg.voronoi import voronoi, voronoi_frames


In [ ]:
# as this is a large, census-tract level dataset, data has been pre-selected for the state of Minnesota
# future revisions will be able to operate on any state and county

svi = gpd.read_file("/Users/brianstrock/Downloads/Minnesota/SVI2018_MINNESOTA_tract.shp")
svi = svi.to_crs("EPSG:4326")
svi = svi.loc[svi['COUNTY'] == "Ramsey"]

In [ ]:
# geocoded from addresses via ESRI geocoding service
# also pre-selected for state of Minnesota

tri = gpd.read_file("/Users/brianstrock/GIS/579Lab5/tri_mn.shp")
tri['trifd'] = tri['trifd'].astype(str)
tri = tri.reset_index().set_index('trifd')
tri = tri.loc[tri["subregion"] == "Ramsey County"]
triKey = tri.index.to_list()

In [ ]:
# this is a HUGE dataset- 20k rows, 185 columns.  As a result, columns of interest are extracted at import.

fields = ['total_air_emissions', 'total_surface_water_discharge', 'on_site_landfills_release_pounds', 'on_site_underground_inj_release_pounds', 'trifd', 'cas_number']

triReleases = pd.read_csv("~/Downloads/tri-releases.csv",
                          encoding="utf-8",
                          usecols=fields
                          )

# dtype conversion, fill na values to avoid errors calculating on np.nan down the road

triReleases['trifd'] = triReleases['trifd'].astype(str)
triReleases = triReleases.fillna(0)

# our X variable of interest will be total releases from site- sum of all emissions types
# emissions types:  air, surface water, underground injection, on-site landfill
# tri data includes both release and transfer data- transfer data is omitted under the assumption
# that transfers from one site may be released at another, thus causing duplication

triReleases['total_emissions'] = triReleases['total_air_emissions'].astype(float) + triReleases['total_surface_water_discharge'].astype(float) + triReleases['on_site_landfills_release_pounds'].astype(float) + triReleases['on_site_underground_inj_release_pounds'].astype(float)
triReleases = triReleases[triReleases.trifd.isin(triKey)]

In [ ]:
# this dataset indexes TRI-regulated chemicals by their known effects

chemicals = pd.read_csv("~/Downloads/chemical-effects.csv",
                        encoding="utf-8",
                        sep=',',
                        index_col='cas'
                        )

# change this to change chemical family of interest
chemIG = chemicals.loc[chemicals['osha_carcinogens'] == 'X']  # select only carcinogens

chemIG = pd.Series(chemIG.osha_carcinogens,
                   index=chemIG.index,
                   name='chemicals: interest group (CAS)'
                   )

# this list serves as a key which will filter TRI sites based on chemical family of interest

chemIGkey = chemIG.index.to_list()

In [ ]:
# As the EPA points out in the TRI dataset, not all emissions are created equal...
# some compounds, like dioxins, are extremely toxic at miniscule doses, while others rise
# to the level of reportable, but require relatively large concentrations to cause effect.

# Furthermore, toxicity can be influenced by factors such as exposure over time and ingestion route.
# To address this, the EPA has created TOXICITY WEIGHTS for all TRI-regulated chemicals.
# These weights act as scalar values which allow for relative toxicity to be accounted for when comparing
# releases.  More details available through EPA's TRI Methodology documentation.

# In this model, our response variable will be WEIGHTED EMISSIONS, IE emissions * toxicity weight
# NOTE:  Toxicity weight chosen is the highest of 4 possible options, which vary based on ingestion route, etc.
# Future versions could account for proper selection of toxicity weights based on site and release characteristics.

tox = pd.read_csv("~/Downloads/rsei_toxicity.csv",
                  encoding='utf-8',
                  index_col="CASNumber"
                  )

tox['MaxTW'] = tox['MaxTW'].astype(float) # dtype conversion

tox = tox[tox.index.isin(chemIGkey)]

tox = pd.Series(index=tox.index,
                data=tox['MaxTW']
                )

# this dictionary is our key to locating the correct scalar value for a given toxic chemical
toxDict = tox.to_dict()

In [ ]:
triReleases = triReleases[triReleases['cas_number'].isin(chemIGkey)]

In [ ]:
# at the very least, I plan to update this so that it's operating off of list comprehensions.
# Anyway, the purpose of this code is to calculate WEIGHTED EMISSIONS.

siteList = []
weightList = []

for row in triReleases.iterrows():
    site = row[1]['trifd']
    cas = row[1]['cas_number']
    emissions = row[1]['total_emissions']
    if cas in toxDict.keys():
        weighted = emissions * toxDict[cas]
        weightList.append(weighted)
        siteList.append(site)
    else:
        weightList.append(emissions)
        siteList.append(site)

In [ ]:
# As we're using weighted X metrics (population demographics), our Y variable is weighted as percent of total

releaseIG = pd.Series(index=siteList, data=weightList, name='weighted_emissions')
releaseIG = releaseIG.fillna(0)
releaseIG = releaseIG.groupby([releaseIG.index]).sum()
releaseIG = pd.Series(releaseIG)
releaseIG = releaseIG.div(np.sum(releaseIG))

In [ ]:
triIG = pd.merge(tri, releaseIG, left_index=True, right_index=True)

In [ ]:
# First, we examine the distribution of our Y variable (weighted emissions)

yVar = triIG['weighted_emissions']

fig = sns.kdeplot(x=yVar)
plt.title('Weighted Emissions in Area of Interest, KDE Curve')
plt.show()

In [ ]:
# this plot charts the kernel density of TRI sites, and weights TRI site point size by % of weighted emissions

ax = gplt.kdeplot(triIG,
                  clip=svi,
                  shade=True,
                  alpha=.8,
                  cmap='inferno',
                  figsize=(10, 5),
                  kwargs={'thresh':'.1'}
                  )
gplt.polyplot(svi,
              ax=ax,
              linewidth=.5,
              facecolor="black",
              edgecolor='white'
              )

gplt.pointplot(triIG,
               color='white',
               ax=ax,
               scale=triIG['weighted_emissions'],
               limits=(1, 10),
               legend=True,
               legend_kwargs= {'title': 'Points: TRI Sites\n% of Weighted Emissions', 'facecolor': 'white', 'loc': 'lower left', 'fancybox': 'True'},)

ax.set_title("Kernel Density Estimate, TRI Sites in Ramsey County, MN (TRI 2018, EPA)",
             fontsize=16,
             color='black')

plt.show()

In [ ]:
# includes Moran's I calculation to quantify spatial autocorrelation in each variable

xCandidates = [svi.EP_MINRTY, svi.EP_GROUPQ, svi.EP_DISABL, svi.EP_POV]  # variables of interest

colorDict = {0: 'OrRd', 1:'PuRd', 2:'GnBu', 3:'PuBu'}  # consistent color scheme across plots


choroCount = 0  # this counter infers the correct colormap

# calculate moran's values for display with choropleth maps

w = lps.weights.Queen.from_dataframe(svi)  # uses queen weights for moran's
w.transform = 'r'  # I'm told this is important

moranList = [Moran(cand, w) for cand in xCandidates]  # calculates Moran's for all 4 variables

# 4 variables, 4 plots, 2x2 grid

fig, axs = plt.subplots(2,2)
fig.set_size_inches(11,8)

fig.suptitle("Vulnerability Characteristics, Ramsey County, MN (SVI 2018, CDC)",
             fontsize=20,
             fontdict={'color': 'white'})


# this loop generates all 4 choropleths and adds them to the figure

for cand, ax in zip(xCandidates, axs.flatten()):

    scheme = mcs.NaturalBreaks(cand, k=5)  # since we're plotting 4 choropleths, Natural Breaks will highlight groupings

    gplt.choropleth(
                        svi,
                        hue=cand,
                        scheme=scheme,
                        cmap=colorDict[choroCount],
                        linewidth=.5,
                        legend=True,
                        legend_kwargs={'title': 'Percent of Pop.', 'facecolor': 'white', 'loc': 'upper left', 'fancybox': 'True'},
                        ax=ax
                        )
    choroCount += 1

# that's all for the choropleths, now let's label them...

axs[0, 0].set_title("Minority",
                    color='white',
                    fontsize=14)

axs[0, 0].annotate("Moran's I: {}\nP-Value: {}".format(round(moranList[0].I, 2), moranList[0].p_sim),
                   xy=(.99, .75),
                   xytext=(.9, .75),
                   xycoords='axes fraction',
                   textcoords='axes fraction',
                   color='white')

axs[0, 1].set_title("Living in Group Quarters", color='white', fontsize=14)
axs[0, 1].annotate("Moran's I: {}\nP-Value: {}".format(round(moranList[1].I, 2), moranList[1].p_sim),
                   xy=(.99, .75),
                   xytext=(.9, .75),
                   xycoords='axes fraction',
                   textcoords='axes fraction',
                   color='white')


axs[1, 0].set_title("Disabled (over age 5)", color='white', fontsize=14)
axs[1, 0].annotate("Moran's I: {}\nP-Value: {}".format(round(moranList[2].I, 2), moranList[2].p_sim),
                   xy=(.99, .75),
                   xytext=(.99, .75),
                   xycoords='axes fraction',
                   textcoords='axes fraction',
                   color='white')

axs[1, 1].set_title("Living in Poverty", color='white', fontsize=14)
axs[1, 1].annotate("Moran's I: {}\nP-Value: {}".format(round(moranList[3].I, 2), moranList[3].p_sim),
                   xy=(.99, .75),
                   xytext=(.99, .75),
                   xycoords='axes fraction',
                   textcoords='axes fraction',
                   color='white')

fig.set_facecolor('black')
fig.show()

In [ ]:
# Aha, how to address the problem in this method:  comparing point and polygon objects in the context of GWR.
# To provide some gauge of how TRI sites relate to the demographics nearby, the TRI sites are used to generate
# voronoi polygons, which contain all points closest to that TRI site.

x = triIG.geometry.x.values
y = triIG.geometry.y.values
triCoords = list(zip(x, y))

trifd = pd.Series(triIG.index.to_list(),
                  name='trifd')

triX = pd.Series(triIG.weighted_emissions.to_list(),
                 name='weighted_emissions')

regionDF, pointDF = voronoi_frames(triCoords,
                                   clip=svi.geometry.unary_union)

regionDF.crs = 'EPSG:4326'

# the double-merge is ugly, but geopandas and .join are fickle, or I just need more practice.
# in any case, these statements merge trifd (site identifier) and values back into the polygons.

triVoronoi = pd.merge(regionDF, trifd,
                      left_index=True,
                      right_index=True)

triVoronoi = pd.merge(triVoronoi, triX,
                      left_index=True,
                      right_index=True)

# Once we have these polygons loaded with the Y values, it's time to align our X data.
# NOTE:  The spatial join here is a bit rough in terms of representing underlying population distributions.
# A much more sophisticated analysis is possible, along the lines of using the underlying population counts
# to assign population estimates to voronoi regions based on proportional overlapping area, etc.
# as it is, the model proceeds with a rougher vision in the hopes of future refinements.

analysisLayer = gpd.sjoin(triVoronoi,
                          svi,
                          how='left',
                          op='contains')

# the join method preserves data from both objects, however it also duplicates each polygon in the process.
# dissolving via mean allows these values to be collapsed into a singular representative geometry.

analysisLayer = analysisLayer.dissolve(by=analysisLayer.trifd,
                                       aggfunc='mean')

analysisLayer = analysisLayer.fillna(0)  # deals with np.nan issues

In [ ]:
# Since we're changing geometries, some additional visualization is helpful.

scheme = mcs.Quantiles(analysisLayer.weighted_emissions)

ax = gplt.choropleth(
    analysisLayer,
    hue=analysisLayer.weighted_emissions,
    scheme=scheme,
    cmap='Reds',
    linewidth=.5,
    legend=True,
    legend_kwargs={'title': 'Percent of Weighted Emissions', 'facecolor': 'white', 'loc': 'lower left', 'fancybox': 'True'},
)
ax.set_title("Weighted Emissions (Percent of Total)\nQuantiles Classification", size=20)
plt.show()

In [ ]:
choroCount = 0  # reset
xCandidates = [analysisLayer.EP_MINRTY, analysisLayer.EP_GROUPQ, analysisLayer.EP_DISABL, analysisLayer.EP_POV]  # variables of interest

fig, axs = plt.subplots(2,2)
fig.set_size_inches(11,8)

fig.suptitle("Vulnerability Characteristics, Voronoi Dissolve (SVI 2018, CDC)",
             fontsize=20,
             fontdict={'color': 'white'})

for cand, ax in zip(xCandidates, axs.flatten()):

    scheme = mcs.NaturalBreaks(cand, k=5)  # since we're plotting 4 choropleths, Natural Breaks will highlight groupings

    gplt.choropleth(
                        analysisLayer,
                        hue=cand,
                        scheme=scheme,
                        cmap=colorDict[choroCount],
                        linewidth=.5,
                        legend=True,
                        legend_kwargs={'title': 'Percent of Pop.', 'facecolor': 'white', 'loc': 'upper left', 'fancybox': 'True'},
                        ax=ax
                        )
    choroCount += 1

# that's all for the choropleths, now let's label them...

axs[0, 0].set_title("Minority",
                    color='white',
                    fontsize=14
                    )

axs[0, 1].set_title("Living in Group Quarters",
                    color='white',
                    fontsize=14
                    )

axs[1, 0].set_title("Disabled (over age 5)",
                    color='white', fontsize=14
                    )

axs[1, 1].set_title("Living in Poverty",
                    color='white',
                    fontsize=14
                    )

fig.set_facecolor('black')
fig.show()

In [ ]:
# At last!  Extract centroids, compute values, regress, report, awesome.

centroids = analysisLayer.centroid  # center of voronoi region = xy of TRI site

y = analysisLayer['weighted_emissions'].values.reshape((-1, 1))  # normalize values
X = analysisLayer[['EP_MINRTY', "EP_GROUPQ", "EP_DISABL", 'EP_POV']].values # predictors
u = centroids.x.values
v = centroids.y.values
triCoords = list(zip(u, v))

X = (X - X.mean(axis=0)) / X.std(axis=0)  # normalizing

y = (y - y.mean(axis=0)) / y.std(axis=0)  # normalizing

gwrSelector = Sel_BW(triCoords, y, X)  # bandwidth selection
gwrBW = gwrSelector.search(bw_min=2)
print('bandwidth: {}'.format(gwrBW))  # report bandwidth

gwrResults = GWR(triCoords,
                 y,
                 X,
                 gwrBW,
                 kernel='gaussian',
                 spherical=True
                 ).fit()  # huzzah!  a regression!

print(gwrResults.summary())  # results summary.  compare output to subsequent coefficent maps to assess validity.

# coefficient capture
int = analysisLayer['gwr_int'] = gwrResults.params[:, 0]
minority = analysisLayer['gwr_min'] = gwrResults.params[:, 1]
groupQ = analysisLayer['gwr_groupq'] = gwrResults.params[:, 2]
disabl = analysisLayer['gwr_disabl'] = gwrResults.params[:, 3]
poverty = analysisLayer['gwr_pov'] = gwrResults.params[:, 4]



#residuals capture
triIG['resid'] = gwrResults.resid_response


# Create scalar mappable for colorbar and stretch colormap across range of data values

coefCandidates = [minority, groupQ, disabl, poverty]

coefMins = [np.min(i) for i in coefCandidates]
coefMaxs = [np.max(i) for i in coefCandidates]

vMin = np.min(coefMins)
vMax = np.max(coefMaxs)

cmap = plt.cm.coolwarm
sm = plt.cm.ScalarMappable(cmap=cmap,
                           norm=plt.Normalize(
                               vmin=vMin,
                               vmax=vMax
                                )
                           )

 #%%  Finally, we plot coefficents.  The residuals are depicted by TRI site point size.

fig, axs = plt.subplots(2, 2)
fig.set_size_inches(11, 8)

fig.suptitle("GWR Model Parameters:  Coefficient values, residuals (point size)",
             fontsize=20,
             color='black'
             )

coefFlag = 0

coefCandidates = [minority, groupQ, disabl, poverty]

for coef, ax in zip(coefCandidates, axs.flatten()):

    scheme = mcs.NaturalBreaks(coef,
                               k=5)

    gplt.pointplot(
        triIG,
        scale=triIG['resid'],
        color='black',
        limits=(1, 20),
        ax=ax,
    )

    gplt.choropleth(
                    analysisLayer,
                    hue=coef,
                    scheme=scheme,
                    cmap=cmap,
                    linewidth=.5,
                    ax=ax,
                    alpha=.8,
                    )

axs[0, 0].set_title("Minority",
                    color='black',
                    fontsize=14)

axs[0, 1].set_title("Living in Group Quarters",
                    color='black',
                    fontsize=14)

axs[1, 0].set_title("Disabled (over age 5)",
                    color='black',
                    fontsize=14)

axs[1, 1].set_title("Living in Poverty",
                    color='black',
                    fontsize=14)


# Set figure options and plot
cax = fig.add_axes([0.9, 0.1, 0.02, 0.75])

cbar = fig.colorbar(sm,
                    cax=cax)

cbar.ax.tick_params(labelsize=14)
fig.tight_layout()

plt.show()

In [ ]:
ax = sns.displot(data=triIG['resid'])
plt.show

In [ ]:
for cand in xCandidates:

    sns.residplot(y, cand, color='blue')
    plt.show
